In [3]:
import cv2 as cv
import numpy as np
import os

In [4]:
#hard-coded grid and score-road
grid_multipliers = [
 [5, 0, 0, 4, 0, 0, 0, 3, 0, 0, 0, 4, 0, 0, 5],
 [0, 0, 3, 0, 0, 4, 0, 0, 0, 4, 0, 0, 3, 0, 0],
 [0, 3, 0, 0, 2, 0, 0, 0, 0, 0, 2, 0, 0, 3, 0],
 [4, 0, 0, 3, 0, 2, 0, 0, 0, 2, 0, 3, 0, 0, 4],
 [0, 0, 2, 0, 1, 0, 1, 0, 1, 0, 1, 0, 2, 0, 0],
 [0, 4, 0, 2, 0, 1, 0, 0, 0, 1, 0, 2, 0, 4, 0],
 [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
 [3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3],
 [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
 [0, 4, 0, 2, 0, 1, 0, 0, 0, 1, 0, 2, 0, 4, 0],
 [0, 0, 2, 0, 1, 0, 1, 0, 1, 0, 1, 0, 2, 0, 0],
 [4, 0, 0, 3, 0, 2, 0, 0, 0, 2, 0, 3, 0, 0, 4],
 [0, 3, 0, 0, 2, 0, 0, 0, 0, 0, 2, 0, 0, 3, 0],
 [0, 0, 3, 0, 0, 4, 0, 0, 0, 4, 0, 0, 3, 0, 0],
 [5, 0, 0, 4, 0, 0, 0, 3, 0, 0, 0, 4, 0, 0, 5]]

score_road = [
    -1, 1, 2, 3, 4, 5, 6, 0, 2, 5, 3, 4, 6, 2, 2, 0, 3, 5, 4, 1, 6, 2, 4, 5, 5, 0, 6, 3, 4, 2, 0, 1, 5, 1, 3, 4,
    4, 4, 5, 0, 6, 3, 5, 4, 1, 3, 2, 0, 0, 1, 1, 2, 3, 6, 3, 5, 2, 1, 0, 6, 6, 5, 2, 1, 2, 5, 0, 3, 3, 5, 0, 6,
    1, 4, 0, 6, 3, 5, 1, 4, 2, 6, 2, 3, 1, 6, 5, 6, 2, 0, 4, 0, 1, 6, 4, 4, 1, 6, 6, 3, 0 ] 


In [5]:
def showimg(img): #function to show the image
    image = np.copy(img)
    image = cv.resize(image,(500,500), interpolation = cv.INTER_AREA)
    cv.imshow("imagine", image)
    cv.waitKey(0)
    cv.destroyAllWindows()

In [6]:
def leeAlgorithm(matrix, start): #bfs to find closest edge of the border
    
    (x, y)=start
    if matrix[x][y] < 255:
        return [y, x]
    
    qe = []
    qe.append((x, y))
    visited = set()
    visited.add((x, y))
    
    height = len(matrix[0])
    length = len(matrix)
    
    dx=[1, 0, -1, 0]
    dy=[0, 1, 0, -1]
    
    while qe:
        x, y = qe.pop(0)
        
        # Explore the neighboring nodes
        for k in range(4):
            nx = x+dx[k]
            ny = y+dy[k]
            if nx>=0 and ny>=0 and nx<length and ny<height:
                if (nx, ny) not in visited:
                    visited.add((nx, ny))
                    qe.append((nx, ny))
                if matrix[nx][ny] < 255:
                    return [ny, nx]
    

In [ ]:
def Connect4BFS(matrix, start): #bfs to find closest edge of the border from the start point
    
    (x, y)= start
    if matrix[x][y] == 1:
        return [y, x]
    
    qe = [(x, y)]
    visited = set()
    visited.add((x, y))
    
    height = 640
    length = 640
    
    dx=[1, 0, -1, 0]
    dy=[0, 1, 0, -1]
    
    while qe:
        x, y = qe.pop(0)
        
        # Explore the neighboring nodes
        for k in range(4):
            nx = x+dx[k]
            ny = y+dy[k]
            if 0 <= nx < length and 0 <= ny < height:
                if (nx, ny) not in visited:
                    visited.add((nx, ny))
                    qe.append((nx, ny))
                if matrix[nx][ny] < 255:
                    return [ny, nx]
    

In [ ]:
def Connect4GetBoard(image,matrix): #stretch image so that the border of the playable board fits in a 700x600 rectangle
    ul= Connect4BFS(matrix, (0,0)) #left up
    ur= Connect4BFS(matrix, (0,639)) #right up
    dr= Connect4BFS(matrix, (639,639)) #right down
    dl= Connect4BFS(matrix, (639,0)) #left down
    
    src = np.array([ul, ur, dr, dl], dtype="float32")
    dst = np.array([[0, 0], [0, 700], [600, 700],[600, 0] ], dtype="float32")
    m = cv.getPerspectiveTransform(src, dst)
    result = cv.warpPerspective(image, m, (700, 600))
    # result = cv.rotate(result, cv.ROTATE_90_CLOCKWISE)
    # result = cv.flip(result, 1)
    
    return result

In [7]:
def transformImg(binimg,img): #stretch image so that the border of the playable board fits in a perfect square
    height= binimg.shape[0]
    width= binimg.shape[1]
    
    #start the bfs from all corners to take the 4 edges of the border from the binary img
    ul= leeAlgorithm(binimg, (0,0)) #left up
    ur= leeAlgorithm(binimg, (0,width-1)) #right up
    dr= leeAlgorithm(binimg, (height-1,width-1)) #right down
    dl= leeAlgorithm(binimg, (height-1,0)) #left down
    
    src = np.array([ul, ur, dr, dl], dtype="float32")
    dst = np.array([[0, 0], [0, 1200], [1200, 1200],[1200, 0] ], dtype="float32")
    M = cv.getPerspectiveTransform(src, dst)
    result = cv.warpPerspective(img, M, (1200, 1200))
    result = cv.rotate(result, cv.ROTATE_90_CLOCKWISE)
    result = cv.flip(result, 1)
    
    return result

In [8]:
def process(path): #here the image will be resized, preprocessed and stretched  
    img=cv.imread(path)
    img=img[1200:2880, 770:2440]
    
    #preprocess: grayscale, blur, sharpen
    img = cv.cvtColor(img, cv.COLOR_BGR2GRAY) #grayscale
    img = cv.GaussianBlur(img, (7, 7), 3) #blur
    sharpen_kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]]) #sharpen
    img = cv.filter2D(img,  -50, sharpen_kernel)
    
    #binarization of a copy of the image 
    binimg = np.copy(img)
    binimg = cv.GaussianBlur(binimg, (3, 3), 3)
    _, binimg = cv.threshold (binimg, 37, 255, cv.THRESH_BINARY) #harsh thresh to get the frame only
    
    result = transformImg(binimg,img) #stretch
    
    return result

In [9]:
#detect pieces
def detectDifference(old_img, new_img):
    differ = np.copy(old_img) # make a new image to get the new piece, which will be the difference between pieces
    for i in range(1200):
        for j in range(1200):
            differ[i][j] = abs(int(new_img[i][j]) - int(old_img[i][j])) #the difference will be the new piece
            
    #process the image made by subtracting old img from new img 
    differ = cv.medianBlur(differ,5)
    differ = cv.GaussianBlur(differ, (0, 0), 8)
    sharpen_kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
    differ = cv.filter2D(differ, -50, sharpen_kernel)
    
    #binarization
    differ_copy = np.copy(differ)
    differ_copy = cv.medianBlur(differ_copy,3)
    differ_copy=cv.GaussianBlur(differ_copy, (1, 1), 5)
    _, differ_copy = cv.threshold (differ_copy, 58, 255, cv.THRESH_BINARY)
    return differ_copy

In [10]:
def otherHalf(i,j,binimg,grid):
    dx = [1, 0, -1, 0] #down right up left
    dy = [0, 1, 0, -1]
    for k in range(4):
        if (i+dx[k]>-1 and i+dx[k]<15 and j+dy[k]>-1 and j+dy[k]<15):
            if grid[i+dx[k]][j+dy[k]] == 0:
                mean_intensity = cv.mean(binimg[(i+dx[k])*80:(i+1+dx[k])*80, (j+dy[k])*80:(j+1+dy[k])*80])[0]
                if mean_intensity>50:  
                    grid[i+dx[k]][j+dy[k]] = 1
                    return (i+dx[k],j+dy[k])  
            
    return 0,0  #return something if nothing is found

In [11]:
def newPiecePos(binimg,grid):
    #every cell in the grid
    for i in range (15):
        for j in range (15):
            #calculate the mean of the pixel values in the cell
            if grid[i][j] == 0:
                mean_intensity = cv.mean(binimg[i*80:(i+1)*80, j*80:(j+1)*80])[0] #calculate the mean 
                if mean_intensity > 50: 
                    grid[i][j] = 1
                    i1,j1 = otherHalf(i,j,binimg,grid)
                    return[i,j,i1,j1]
                
    return [0,0,0,0]  #return something if nothing is found

In [12]:
def detectNumber(i,j,img):
    #cut out the cell with the (half of) piece we need from the big image
    src = img[i*80:(i+1)*80,j*80:(j+1)*80] 

    aux = cv.bitwise_not(src)
    _, aux = cv.threshold (aux, 113, 255, cv.THRESH_BINARY)
    
    # Morph open 
    kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE, (5,5))
    opening = cv.morphologyEx(aux, cv.MORPH_OPEN, kernel, iterations=1)
    
    # Find contours and filter using contour area and aspect ratio
    cnts = cv.findContours(opening, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    if len(cnts) == 2:
        cnts = cnts[0]
    else:
        cnts = cnts[1]
        
    cnt = 0
    for c in cnts:
        perimeter = cv.arcLength(c, True)
        cont_aprox = cv.approxPolyDP(c, 0.02 * perimeter, True)
        area = cv.contourArea(c)
        if len(cont_aprox) > 5 and area > 200 and area < 350:            
            ((_, _), r) = cv.minEnclosingCircle(c)
            if r<15:
                cnt+=1
    return cnt

In [13]:
def getMoves(moves_path) :  #function that will get the order or player moves out of the mutari.txt file
    
    moves_array = np.zeros(20, dtype=int) 
    
    with open(moves_path, 'r') as moves:
        for i in range (20):
                turn = moves.readline() 
                turn = int(turn[15])-1 
                moves_array[i] = turn
                
    return moves_array

In [14]:
def computeScore(i1,j1,i2,j2,nb1,nb2,player,moves_path,idx):
    
    moves = getMoves(moves_path)
    
    #compute score
    round_score = grid_multipliers[i1][j1] + grid_multipliers[i2][j2] #add the multipliers found in the matrix at i,j
        
    if(nb1 == nb2): #double multiplier score if it's a double domino
        round_score *= 2
    if(nb1 == score_road[player[moves[idx]]] or nb2 == score_road[player[moves[idx]]]): #if the player has the pawn on a number form the domino piece
        round_score+=3
    if(nb1 == score_road[player[1-moves[idx]]] or nb2 == score_road[player[1-moves[idx]]]): #check for the other player too
        player[1-moves[idx]]+=3
            
    player[moves[idx]] += round_score #add the calculated score
        
    return (round_score)

In [15]:
def runProject(folder_path,result_path): 
    #initializing our variables
    grid = np.zeros((15,15)) 
    ant_file= process("imagini_auxiliare/01.jpg") 
    player = [0, 0] 
    moves_path_idx = 1 
    player_moves_idx = -1 
    
    #setting up paths
    files=os.listdir(folder_path) 
    directory_path = result_path + "/R_I"
    if not os.path.exists(directory_path):
        os.makedirs(directory_path)
    moves_path = folder_path + "/1_mutari.txt" 

    
    #parsing through files
    for file in files:
        if file[-3:] == 'jpg':
            
            player_moves_idx = player_moves_idx + 1

            #apply all functions
            img = process(folder_path+'/'+file)  
            diff = detectDifference(ant_file,img)
            i1,j1,i2,j2 = newPiecePos(diff,grid)   
            nb1 = detectNumber (i1, j1, img)
            nb2 = detectNumber (i2, j2, img)
            round_score = computeScore(i1,j1,i2,j2,nb1,nb2,player,moves_path,player_moves_idx)

            #name, create and write in file
            file_path = file[:-3] + "txt" 
            full_path = os.path.join(directory_path, file_path) 
            with open(full_path, 'w') as f:
                f.write("{}{} {}\n".format(i1+1,chr(j1+65), nb1)) 
                f.write("{}{} {}\n".format(i2+1,chr(j2+65), nb2))
                f.write(str(round_score)) 

            #reinitialize variables when we reach the end of a game
            if file[-6:] == '20.jpg': 
                moves_path_idx += 1 
                moves_path = folder_path +"/" + str(moves_path_idx) + "_mutari.txt"
                player_moves_idx = -1 
                player [0] = player [1] = 0
                grid = np.zeros((15,15)) 
                ant_file = process("imagini_auxiliare/01.jpg")
            else:
                ant_file = img  

In [17]:
runProject ('ground truth','rezultate')